# Задача 1.

Рассчитать с машинной точностью интеграл с неопределенностью 
 квадратурой на разностных схемах.

In [2]:
from decimal import Decimal, getcontext
import math
from functools import lru_cache
import numpy as np

###############################################################################
#  1) Вспомогательные функции для разностных квадратур                        #
###############################################################################
getcontext().prec = 100  # повышенная точность для Decimal

@lru_cache(None)
def factorial_dec(n: int) -> Decimal:
    if n < 0:
        raise ValueError("Negative factorial not defined")
    if n == 0 or n == 1:
        return Decimal(1)
    return Decimal(n) * factorial_dec(n - 1)

@lru_cache(None)
def comb_dec(n: int, k: int) -> Decimal:
    if k < 0 or k > n:
        return Decimal(0)
    return factorial_dec(n) / (factorial_dec(k) * factorial_dec(n - k))

@lru_cache(None)
def D(n: int, j: int) -> Decimal:
    """
    Вычисляет D_n^(j) по заданной рекуррентной формуле (см. теорию).
    """
    if j == 0:
        return Decimal(1)
    # "прогреваем" кэш:
    _ = D(n, j-1)
    return _compute_all_D_up_to(n, j)[j]

@lru_cache(None)
def _compute_all_D_up_to(n: int, J: int):
    results = [Decimal(0)]*(J+1)
    results[0] = Decimal(1)
    for j in range(J):
        s = Decimal(0)
        for k in range(n + 2*j + 1):
            for l in range(j+1):
                val = (
                    Decimal((-1)**k)
                    * results[l]
                    * comb_dec(n+2*l, k - (j - l))
                    * Decimal((n + 2*j - 2*k)**(n + 2*j + 2))
                    / (Decimal(2)**(n + 2*j + 2) * factorial_dec(n + 2*j + 2))
                )
                s += val
        # Финальная формула берёт только s, без + results[j].
        results[j+1] = s
    return results

@lru_cache(None)
def A(k: int, n: int, m: int) -> Decimal:
    """
    A_{k,n}^m = sum_{l=0}^m [ (-1)^(k-m) * D_n^(l) * C_{n+2l}^{k - m + l} ].
    """
    s = Decimal(0)
    sign = Decimal((-1)**(k - m))
    for l in range(m+1):
        s += (
            sign
            * D(n, l)
            * comb_dec(n+2*l, k - m + l)
        )
    return s

@lru_cache(None)
def W_list(m: int):
    """
    Коэффициенты W_k^m (k=0..2m) в Decimal.

      W_k^m = sum_{n=0}^m [ A_{k,2n}^{m-n} / (2^(2n)*(2n+1)!) ].
    """
    result = []
    for k in range(2*m + 1):
        s = Decimal(0)
        for n_ in range(m+1):
            a_val = A(k, 2*n_, m - n_)
            denom = (Decimal(2)**(2*n_) * factorial_dec(2*n_+1))
            s += a_val / denom
        result.append(s)
    return result

def build_function_values_diff_scheme(f, a: float, b: float, J: int, m: int):
    """
    Центральная сетка: x_j = a + (j+0.5)*h, j=-m..(J-1+m).
    Возвращает (y_values, h, calls).
    """
    h = (b - a)/J
    values = []
    for j_real in range(-m, (J - 1) + m + 1):
        x_val = a + (j_real + 0.5)*h
        values.append(f(x_val))
    calls = len(values)
    return values, h, calls

def integrate_by_diff_scheme(f, a: float, b: float, J: int, m: int):
    """
    Разностная (дифференсная) квадратура порядка 2m:
      ∫[a..b] f(x) dx ≈ h Σ_j Σ_k [ W_{m-k} * f( x_{j+k} ) ].
    Возвращает (approx, calls).
    """
    y_values, h, calls = build_function_values_diff_scheme(f, a, b, J, m)
    W_dec = W_list(m)
    W = [float(wd) for wd in W_dec]
    
    offset = m
    total_sum = 0.0
    for j in range(J):
        loc_sum = 0.0
        for k in range(-m, m+1):
            idx = (j + k) + offset
            loc_sum += W[m - k] * y_values[idx]
        total_sum += loc_sum
    return (h * total_sum, calls)

###############################################################################
#        2) Метод Симпсона (классический) с заданным числом вызовов f         #
###############################################################################

def build_function_values_simpson(f, a: float, b: float, n: int):
    h = (b - a)/n
    y = [f(a + i*h) for i in range(n+1)]
    return y, h

def simpson_by_calls(f, a: float, b: float, calls: int):
    """
    Метод Симпсона «под число вызовов calls».
    Если (calls-1) нечётно, уменьшаем n на 1.
    Если n < 2, берём n=2.
    Возвращает (approx, real_calls).
    """
    n = calls - 1
    if n % 2 == 1:
        n -= 1
    if n < 2:
        n = 2
    
    y, h = build_function_values_simpson(f, a, b, n)
    real_calls = n + 1
    
    # Классическая формула Симпсона
    s_odd = 0.0
    s_even = 0.0
    for i in range(1, n, 2):
        s_odd += y[i]
    for i in range(2, n, 2):
        s_even += y[i]
    simpson_approx = (h/3.0)*(y[0] + y[n] + 4.0*s_odd + 2.0*s_even)

    return (simpson_approx, real_calls)

###############################################################################
#                             3) Основная часть                                #
###############################################################################

def main():
    # ------------------------------------------------------
    # Пример 1: f1(x)= sin(x/2)/(e^x-1) на [-1,1].
    # Нет простого закрытого выражения; оставляем без аналитической формулы.
    # ------------------------------------------------------
    def f1(x: float) -> float:
        if x==0:
            return (float)(1/2)
        return math.sin(x/2)/(np.exp(x)-1)

    # "Имитация" точного значения с помощью большого числа разбиений Симпсона.
    # (Если хотите исключить численную оценку, уберите пример 1.)
    def get_almost_exact_for_f1():
        big_calls = 10_000_001
        val, _ = simpson_by_calls(f1, -1.0, 1.0, big_calls)
        return val

    exact_val_1 = get_almost_exact_for_f1()

    a1, b1 = -1.0, 1.0
    print("\n=========== Пример 1 ===========")
    print("Интеграл: I1 = ∫[-1,1] sin(x/2)/(e^x-1) dx")
    print("(псевдо-«точное» значение, большое разбиение Симпсона)\n")
    print(exact_val_1)

    for m in [3, 4, 5, 6, 7]:
        print(f"--- m={m} ---")
        for J in [2, 4, 8, 16]:
            diff_approx, diff_calls = integrate_by_diff_scheme(f1, a1, b1, J, m)
            diff_err = abs(diff_approx - exact_val_1)

            simpson_approx, simp_real_calls = simpson_by_calls(f1, a1, b1, diff_calls)
            simpson_err = abs(simpson_approx - exact_val_1)

            print(f" J={J}, calls={diff_calls:2d} | "
                  f"DiffScheme={diff_approx:.8f}, err={diff_err:.2e} | "
                  f"Simpson={simpson_approx:.8f}, err={simpson_err:.2e}  "
                  f"(n={simp_real_calls - 1} subintrvls)")
        print()

    
if __name__ == "__main__":
    main()


=========== Пример 1 ===========
Интеграл: I1 = ∫[-1,1] sin(x/2)/(e^x-1) dx
(псевдо-«точное» значение, большое разбиение Симпсона)

1.0130392362326186
--- m=3 ---
 J=2, calls= 8 | DiffScheme=1.01303490, err=4.34e-06 | Simpson=1.01303303, err=6.20e-06  (n=6 subintrvls)
 J=4, calls=10 | DiffScheme=1.01303922, err=1.96e-08 | Simpson=1.01303728, err=1.95e-06  (n=8 subintrvls)
 J=8, calls=14 | DiffScheme=1.01303924, err=7.95e-11 | Simpson=1.01303885, err=3.84e-07  (n=12 subintrvls)
 J=16, calls=22 | DiffScheme=1.01303924, err=3.06e-13 | Simpson=1.01303919, err=4.97e-08  (n=20 subintrvls)

--- m=4 ---
 J=2, calls=10 | DiffScheme=1.01303816, err=1.07e-06 | Simpson=1.01303728, err=1.95e-06  (n=8 subintrvls)
 J=4, calls=12 | DiffScheme=1.01303923, err=1.45e-09 | Simpson=1.01303844, err=7.98e-07  (n=10 subintrvls)
 J=8, calls=16 | DiffScheme=1.01303924, err=1.53e-12 | Simpson=1.01303903, err=2.07e-07  (n=14 subintrvls)
 J=16, calls=24 | DiffScheme=1.01303924, err=6.22e-15 | Simpson=1.01303920, 

# Задача 2.

Используя методику прямой сшивки при взятии сумм рядов, рассчитать постоянную 
 Эйлера-Маскерони с ошибкой в 13 знаке по формуле 
 


In [ ]:
@lru_cache(None)
def W_list(m: int):
    """
    Коэффициенты W_k^m (k=0..2m) в Decimal.

      W_k^m = sum_{n=0}^m [ A_{k,2n}^{m-n} / (2^(2n)*(2n+1)!) ].
    """
    result = []
    for k in range(2*m + 1):
        s = Decimal(0)
        for n_ in range(m+1):
            a_val = A(k, 2*n_, m - n_)
            denom = (Decimal(2)**(2*n_) * factorial_dec(2*n_+1))
            s += a_val / denom
        result.append(s)
    return result

@lru_cache(None)
def I_list(m: int):
    result = []
    for l in range(2*m+1):
        s = Decimal(0)
        for k in range(l+1):
            s += W_list(m)[2*m-k]
        result.append(s)
    print(result)
    return result

def f(x):
    if x == 1.0:
        return 1
    return 1/x -np.log(x) + np.log(x-1)

def integratePartGamma(j_m):
    # Первообразная f
    return -((j_m-0.5)*np.log(1-1/(j_m+0.5))+1)

def GammaSearch(f_, m, j_m):
    s = 0
    #Integral part
    s += integratePartGamma(j_m)

    print("Integral part = ", s)

    # Partial Summ
    for j in range(1, j_m-m+1):
        s += f_(j)
    print("Partial Sum + Integral Part = ", s)

    # I part
    

    I_dec = I_list(m)
    I = [float(id) for id in I_dec]

    for j in range(j_m-m+1, j_m+m+1):
        
        s += (1-I[j+m-j_m-1])*f_(j)
    
    return(s)
    



In [31]:
Gamma = Decimal(0.5772156649015328606065120900824024310421593359399235988057672348848677267776646709369470632917467495146314472498070824809605040144865428362241739976449235362535003337429373377376739427925952582470949160087352039481656708532331517766115286211995015079847937450857057400299213547861)
j_mList = np.array([10e1, 10e3, 10e5, 10e7])
mList = np.array([5, 7, 9, 11])

for m in mList:
    for j_m in j_mList:

        print(f"m = {m}, j_m = {j_m} :", np.abs(GammaSearch(f, 7, 1000)-float(Gamma)))




Integral part =  -0.0004999167083316047
Partial Sum + Integral Part =  0.5772191883535537
m = 5, j_m = 100.0 : 1.0769163338864018e-14
Integral part =  -0.0004999167083316047
Partial Sum + Integral Part =  0.5772191883535537
m = 5, j_m = 10000.0 : 1.0769163338864018e-14
Integral part =  -0.0004999167083316047
Partial Sum + Integral Part =  0.5772191883535537
m = 5, j_m = 1000000.0 : 1.0769163338864018e-14
Integral part =  -0.0004999167083316047
Partial Sum + Integral Part =  0.5772191883535537
m = 5, j_m = 100000000.0 : 1.0769163338864018e-14
Integral part =  -0.0004999167083316047
Partial Sum + Integral Part =  0.5772191883535537
m = 7, j_m = 100.0 : 1.0769163338864018e-14
Integral part =  -0.0004999167083316047
Partial Sum + Integral Part =  0.5772191883535537
m = 7, j_m = 10000.0 : 1.0769163338864018e-14
Integral part =  -0.0004999167083316047
Partial Sum + Integral Part =  0.5772191883535537
m = 7, j_m = 1000000.0 : 1.0769163338864018e-14
Integral part =  -0.0004999167083316047
Part

# Задача 3.

Вычислить сумму ряда

 
 
с
с точностью 
, применяя метод Куммера

In [42]:
def f(x):
    return (x*x+1)*np.cos(2*x)/(x**4+x*x+1)

def g(x):
    return np.cos(2*x)/x**2

gSumm = np.pi**2/6 - np.pi+1

def CummerRule(f_, g_, i=1, m=10):
    s = 0
    for j in range(i, m+1):
        s += f_(j)-g_(j)
    return s + gSumm

for j in np.array([10, 100, 1000]):
    print(f"Сумма правилом Кумера, k = {j}:", " "*(4-int(np.log10(j))), CummerRule(f, g, 1, j))



Сумма правилом Кумера, k = 10:     -0.35126576395052383
Сумма правилом Кумера, k = 100:    -0.35126538587918177
Сумма правилом Кумера, k = 1000:   -0.35126538587922834


# Задача 4.

Взять предел

 
 
где

  
Поиграться с выбором шага.

In [43]:
import math
from math import factorial

from decimal import Decimal, getcontext
import math
from functools import lru_cache
import numpy as np

###############################################################################
#  1) Вспомогательные функции для разностных квадратур                        #
###############################################################################
getcontext().prec = 100  # повышенная точность для Decimal

@lru_cache(None)
def factorial_dec(n: int) -> Decimal:
    if n < 0:
        raise ValueError("Negative factorial not defined")
    if n == 0 or n == 1:
        return Decimal(1)
    return Decimal(n) * factorial_dec(n - 1)

@lru_cache(None)
def comb_dec(n: int, k: int) -> Decimal:
    if k < 0 or k > n:
        return Decimal(0)
    return factorial_dec(n) / (factorial_dec(k) * factorial_dec(n - k))

@lru_cache(None)
def D(n: int, j: int) -> Decimal:
    """
    Вычисляет D_n^(j) по заданной рекуррентной формуле (см. теорию).
    """
    if j == 0:
        return Decimal(1)
    # "прогреваем" кэш:
    _ = D(n, j-1)
    return _compute_all_D_up_to(n, j)[j]

@lru_cache(None)
def _compute_all_D_up_to(n: int, J: int):
    results = [Decimal(0)]*(J+1)
    results[0] = Decimal(1)
    for j in range(J):
        s = Decimal(0)
        for k in range(n + 2*j + 1):
            for l in range(j+1):
                val = (
                    Decimal((-1)**k)
                    * results[l]
                    * comb_dec(n+2*l, k - (j - l))
                    * Decimal((n + 2*j - 2*k)**(n + 2*j + 2))
                    / (Decimal(2)**(n + 2*j + 2) * factorial_dec(n + 2*j + 2))
                )
                s += val
        # Финальная формула берёт только s, без + results[j].
        results[j+1] = s
    return results

@lru_cache(None)
def A(k: int, n: int, m: int) -> Decimal:
    """
    A_{k,n}^m = sum_{l=0}^m [ (-1)^(k-m) * D_n^(l) * C_{n+2l}^{k - m + l} ].
    """
    s = Decimal(0)
    sign = Decimal((-1)**(k - m))
    for l in range(m+1):
        s += (
            sign
            * D(n, l)
            * comb_dec(n+2*l, k - m + l)
        )
    return s



def y_of_x(x: float) -> float:
    
    if x > 0:
        return np.cosh(np.sqrt(np.abs(x)))
    elif x < 0:
        return np.cos(np.sqrt(np.abs(x)))
    else:
        
        return 2.0

def f_of_t(t: float) -> float:
    
    
    if t==0:
        
        return 0
    return ((y_of_x(t) - 1.0) / t)


def r_of_n(n: int) -> int:
    
    return (n % 2)





def limit_by_sym_scheme(m: int, h: float, f) -> float:
    
    
    N = 2*m + 1

    
    j_min = - (2*m + 1)
    j_max = + (2*m + 1)
    size_y = j_max - j_min + 1  
    offset = - j_min            

    
    
    y_array = [0.0]*size_y
    for j in range(j_min, j_max+1):
        arg = (2*j - 1)*(h/2.0)
        y_array[j + offset] = f(arg)

    
    S = 0.0
    for n_ in range(N+1):  # n_ = 0..N
        rr = r_of_n(n_)   # 0 или 1
        alpha = m - (n_ - rr)/2.0
        
        alpha_int = int(alpha)  

        
        sign = (-1)**n_
        denom = (4**n_) * float(factorial(n_))
        factor_n = sign / denom

        
        max_k = 2*m + rr
        for k_ in range(max_k+1):
        
            j_idx = (2*m + rr) - 2*k_  
        
            y_val = y_array[j_idx + offset]

        
            A_val = float(A(k_, n_, alpha_int))

            S += A_val * factor_n * y_val

    return S


def demo():
    
    def f_for_scheme(t):
        return f_of_t(t)

    
    m_arr = np.array([1+2*j for j in range(5)])

    for m in m_arr:
        print("m =", m)
        for p in [4,5,6,7, 8, 9, 10]:
            h = 10**(-p)
            val = limit_by_sym_scheme(m, h, f_for_scheme)
            print(f"h = 1e-{p},  errorS = {np.abs(val-0.5)}")


demo()


m = 1
h = 1e-4,  errorS = 4.16665319752374e-06
h = 1e-5,  errorS = 4.1667787492594144e-07
h = 1e-6,  errorS = 4.165828942914729e-08
h = 1e-7,  errorS = 4.827648558691777e-09
h = 1e-8,  errorS = 3.038735540972226e-09
h = 1e-9,  errorS = 1.9991783783979145e-08
h = 1e-10,  errorS = 4.137018549954519e-08
m = 3
h = 1e-4,  errorS = 4.1666531853112865e-06
h = 1e-5,  errorS = 4.1667782002541287e-07
h = 1e-6,  errorS = 4.165926109633844e-08
h = 1e-7,  errorS = 4.805699671539543e-09
h = 1e-8,  errorS = 3.0728319333483967e-09
h = 1e-9,  errorS = 1.909064573091257e-08
h = 1e-10,  errorS = 4.137018561056749e-08
m = 5
h = 1e-4,  errorS = 4.1666531820361286e-06
h = 1e-5,  errorS = 4.166778150849204e-07
h = 1e-6,  errorS = 4.165955264090471e-08
h = 1e-7,  errorS = 4.799510511244165e-09
h = 1e-8,  errorS = 3.0910269344097685e-09
h = 1e-9,  errorS = 1.889359813045388e-08
h = 1e-10,  errorS = 4.137018505545598e-08
m = 7
h = 1e-4,  errorS = 4.166653180259772e-06
h = 1e-5,  errorS = 4.1667781525145386e-07
